In [1]:
import modulos as md
import numpy as np
import os
import sys
import constantes as cons
np.seterr(divide='ignore')
lista_filtrada = []
#lista=md.filtradodemonedas ()
lista = ['BTCUSDT', 'ETHUSDT', 'BCHUSDT', 'XRPUSDT', 'EOSUSDT', 'LTCUSDT', 'TRXUSDT', 'ETCUSDT', 'LINKUSDT', 'ADAUSDT', 'BNBUSDT', 'ATOMUSDT', 'NEOUSDT', 'COMPUSDT', 'DOGEUSDT', 'DOTUSDT', 'CRVUSDT', 'TRBUSDT', 'RUNEUSDT', 'SUSHIUSDT', 'SOLUSDT', 'UNIUSDT', 'AVAXUSDT', 'FTMUSDT', 'NEARUSDT', 'AAVEUSDT', 'FILUSDT', 'MATICUSDT', 'SANDUSDT', 'MANAUSDT', 'STMXUSDT', 'MASKUSDT', 'DYDXUSDT', 'GALAUSDT', 'CELOUSDT', 'GMTUSDT', 'APEUSDT', 'OPUSDT', 'INJUSDT', 'APTUSDT', 'FETUSDT', 'RNDRUSDT', 'CFXUSDT', 'ACHUSDT', 'ARBUSDT', 'SUIUSDT', 'WLDUSDT', 'CYBERUSDT', 'ARKUSDT', 'STRAXUSDT', 'LOOMUSDT', 'POLYXUSDT', 'GASUSDT', 'TIAUSDT']
#lista = ['CYBERUSDT']

timeframe = '1h'
if len(lista) > 1:
    imprimo=False
    debug=False
else:
    imprimo= True
    debug=True
print(lista)
porcentajes_sumados=0
win_rate_buenos = 0
win_rate_malos = 0
trades = 0
for symbol in lista:    
    try:
        #print(symbol)
        data = md.estrategia_smart(symbol, debug = False, refinado = False, file_source = False, timeframe = timeframe)
        resultado=md.backtesting_smart(data, plot_flag=imprimo, symbol=symbol)
        if resultado['Win Rate [%]'] >= 50:
            lista_filtrada.append(symbol)
        print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {resultado['Profit Factor']} - Win Rate [%]: {resultado['Win Rate [%]']}")
        porcentajes_sumados=porcentajes_sumados+resultado['Return [%]']
        trades = trades + resultado['# Trades']
        if not np.isnan(resultado['Win Rate [%]']):
            if resultado['Win Rate [%]'] >= 50:
                win_rate_buenos = win_rate_buenos+1
            else:
                win_rate_malos = win_rate_malos+1
        if ((data.Close.iloc[-1] < (data.decisional_alcista_high.iloc[-2] + (data.atr.iloc[-1]*3)))
            or 
            (data.Close.iloc[-1] > (data.decisional_bajista_low.iloc[-2] - (data.atr.iloc[-1]*3)))
            ):
            print(f"posible entrada symbol: {symbol}")
    except Exception as falla:
        _, _, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("\nError: "+str(falla)+" - line: "+str(exc_tb.tb_lineno)+" - file: "+str(fname)+"\n")
        pass
for symbol in lista_filtrada:
    md.printandlog(cons.nombrelog,symbol,pal=1)
print(f"\nTimeframe {timeframe} - porcentajes_sumados {md.truncate(porcentajes_sumados,2)} - trades {trades} - win_rate_buenos {win_rate_buenos} - win_rate_malos {win_rate_malos} - Ganancia por trade: {md.truncate((porcentajes_sumados/trades if trades !=0 else porcentajes_sumados),2)}%")
#(data.loc[data['Open Time'] == '2023-10-03 03:00:00'])


['BTCUSDT', 'ETHUSDT', 'BCHUSDT', 'XRPUSDT', 'EOSUSDT', 'LTCUSDT', 'TRXUSDT', 'ETCUSDT', 'LINKUSDT', 'ADAUSDT', 'BNBUSDT', 'ATOMUSDT', 'NEOUSDT', 'COMPUSDT', 'DOGEUSDT', 'DOTUSDT', 'CRVUSDT', 'TRBUSDT', 'RUNEUSDT', 'SUSHIUSDT', 'SOLUSDT', 'UNIUSDT', 'AVAXUSDT', 'FTMUSDT', 'NEARUSDT', 'AAVEUSDT', 'FILUSDT', 'MATICUSDT', 'SANDUSDT', 'MANAUSDT', 'STMXUSDT', 'MASKUSDT', 'DYDXUSDT', 'GALAUSDT', 'CELOUSDT', 'GMTUSDT', 'APEUSDT', 'OPUSDT', 'INJUSDT', 'APTUSDT', 'FETUSDT', 'RNDRUSDT', 'CFXUSDT', 'ACHUSDT', 'ARBUSDT', 'SUIUSDT', 'WLDUSDT', 'CYBERUSDT', 'ARKUSDT', 'STRAXUSDT', 'LOOMUSDT', 'POLYXUSDT', 'GASUSDT', 'TIAUSDT']
BTCUSDT - Return [%]: 3.27% - # Trades: 2 - Profit Factor: 3.562676835967396 - Win Rate [%]: 50.0
ETHUSDT - Return [%]: -1.43% - # Trades: 1 - Profit Factor: 0.0 - Win Rate [%]: 0.0
BCHUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
XRPUSDT - Return [%]: -2.96% - # Trades: 1 - Profit Factor: 0.0 - Win Rate [%]: 0.0
EOSUSDT - Return [%]: 0.0% - # 